<a href="https://colab.research.google.com/github/lcoia/LearningLangChain/blob/main/Chapter1/Chapter1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain-groq langchain-community

In [ ]:
"""
a-llm.py
Note: using a free Groq model instead of paid OpenAI

https://medium.com/data-engineer-things/bigquerys-ridiculous-pricing-model-cost-us-10-000-in-just-22-seconds-7d52e3e4ae60

"""

from langchain_groq.chat_models import ChatGroq

In [ ]:
# Store your API keys in Google Colab Secrets
from google.colab import userdata

In [ ]:
model = ChatGroq(model="llama3-70b-8192", api_key=userdata.get('GROQ_API_KEY'))

In [ ]:
response = model.invoke("The sky is")
print(response.content)

In [ ]:
"""
b-chat.py

HumanMessage - A message sent from the perspective of the human, with user role.
"""
from langchain_core.messages import HumanMessage
prompt = [HumanMessage("What is the capital of France?")]

In [ ]:
response = model.invoke(prompt)
print(response.content)

In [ ]:
"""
c-system.py

SystemMessage - A message setting the instructions the AI should follow, with the system role.
"""
from langchain_core.messages import SystemMessage

system_msg = SystemMessage(
    "You are a helpful assistant that responds to questions with three exclamation marks."
)
human_msg = HumanMessage("What is the capital of France?")

response = model.invoke([system_msg, human_msg])
print(response.content)

In [ ]:
"""
d-promt.py

PromptTemplate - Making LLM prompts reusable

https://python.langchain.com/v0.1/docs/modules/model_io/prompts/quick_start/
"""
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template("""Answer the question based on the context below.
If the question cannot be answered using the information provided, answer with "I don't know".

Context: {context}

Question: {question}

Answer: """)

prompt = template.invoke(
    {
        "context": "The most recent advancements in NLP are being driven by Large Language Models (LLMs). These models outperform their smaller counterparts and have become invaluable for developers who are creating applications with NLP capabilities. Developers can tap into these models through Hugging Face's `transformers` library, or by utilizing OpenAI and Cohere's offerings through the `openai` and `cohere` libraries, respectively.",
        "question": "Which model providers offer LLMs?",
    }
)

print(prompt)

In [ ]:
"""
e-prompt-model.py

Invoke the model with the prompt
"""
response = model.invoke(prompt)
print(response)

In [ ]:
"""
f-chat-prompt.py

ChatPromptTemplate - Prompt template for chat models.
Note: This example uses the
"""
from langchain_core.prompts import ChatPromptTemplate
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            'Answer the question based on the context below. If the question cannot be answered using the information provided, answer with "I don\'t know".',
        ),
        ("human", "Context: {context}"),
        ("human", "Question: {question}"),
    ]
)

response = template.invoke(
    {
        "context": "The most recent advancements in NLP are being driven by Large Language Models (LLMs). These models outperform their smaller counterparts and have become invaluable for developers who are creating applications with NLP capabilities. Developers can tap into these models through Hugging Face's `transformers` library, or by utilizing OpenAI and Cohere's offerings through the `openai` and `cohere` libraries, respectively.",
        "question": "Which model providers offer LLMs?",
    }
)

print(response)

In [ ]:
"""
g-chat-prompt-model.py
"""
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            'Answer the question based on the context below. If the question cannot be answered using the information provided, answer with "I don\'t know".',
        ),
        ("human", "Context: {context}"),
        ("human", "Question: {question}"),
    ]
)
prompt = template.invoke(
    {
        "context": "The most recent advancements in NLP are being driven by Large Language Models (LLMs). These models outperform their smaller counterparts and have become invaluable for developers who are creating applications with NLP capabilities. Developers can tap into these models through Hugging Face's `transformers` library, or by utilizing OpenAI and Cohere's offerings through the `openai` and `cohere` libraries, respectively.",
        "question": "Which model providers offer LLMs?",
    }
)

print(model.invoke(prompt))

In [ ]:
"""
h-structured.py

Getting specific output formats from the model.
"""
from pydantic import BaseModel


class AnswerWithJustification(BaseModel):
    """An answer to the user's question along with justification for the answer."""

    answer: str
    """The answer to the user's question"""
    justification: str
    """Justification for the answer"""

structured_llm = model.with_structured_output(AnswerWithJustification)
response = structured_llm.invoke(
    "What weighs more, a pound of bricks or a pound of feathers")
print(response)

In [ ]:
"""
https://python.langchain.com/docs/how_to/structured_output/#pydantic-class

Beyond just the structure of the Pydantic class, the name of the Pydantic class, the docstring,
and the names and provided descriptions of parameters are very important.
Most of the time with_structured_output is using a model's function/tool calling API,
and you can effectively think of all of this information as being added to the model prompt.
"""
from typing import Optional
from pydantic import BaseModel, Field


# Pydantic
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )

structured_llm = model.with_structured_output(Joke)
response = structured_llm.invoke("Tell me a joke about cats")
print(response)

In [ ]:
"""
i-csv.py

https://python.langchain.com/api_reference/core/output_parsers.html
"""

from langchain_core.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()

response = parser.invoke("apple, banana, cherry")
print(response)

In [ ]:
# invoke() takes a single input and returns a single output.

completion = model.invoke("What is the capital of France?")
print(completion)


In [ ]:
# batch() takes a list of inputs and returns a list of outputs.

completions = model.batch(["What is the capital of Ohio?", "What is the capital of Spain?"])
print(completions)

In [ ]:
# stream() takes a single input and returns an iterator of parts of the output as they become available.

for token in model.stream("What is the capital of Germany?"):
    print(token)